In [186]:
import pandas as pd
import glob, os, sys
import xml.etree.ElementTree as ET
from tqdm import tqdm_notebook as tqdm
sys.path.append('../scripts/')
import util
import parsing_util
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import HTML, display, display_html
import json
import pyperclip
import numpy as np

In [83]:
stanford_files = glob.glob('../data/news-article-flatlist/stanford-parses/*')
source_summary_output = []
source_sentences_output = []
for idx, stanford_parse in tqdm( enumerate(stanford_files), total=len(stanford_files)):
    
    if idx  > 3:
        break

In [103]:
sent_idx = 5
sent_to_ents = list(zip(sents_list[sent_idx], named_ents_list[sent_idx]))
orgs = get_contiguous_organizations(sent_to_ents)

In [187]:
query_yago_for_organization(orgs[1]).head()

,organization,type,val
0,"{'type': 'uri', 'value': 'http://yago-knowledg...","{'type': 'uri', 'value': 'http://www.w3.org/20...",http://www.w3.org/2002/07/owl#Thing
1,"{'type': 'uri', 'value': 'http://yago-knowledg...","{'type': 'uri', 'value': 'http://yago-knowledg...",/wordnet_geographical_area_108574314
2,"{'type': 'uri', 'value': 'http://yago-knowledg...","{'type': 'uri', 'value': 'http://yago-knowledg...",/wordnet_location_100027167
3,"{'type': 'uri', 'value': 'http://yago-knowledg...","{'type': 'uri', 'value': 'http://yago-knowledg...",/yagoGeoEntity
4,"{'type': 'uri', 'value': 'http://yago-knowledg...","{'type': 'uri', 'value': 'http://yago-knowledg...",/wordnet_physical_entity_100001930


# Look at tagged files

In [113]:
tagged_files = glob.glob('../app/data/*-marke*')
tags = []
for f in tagged_files:
    tags.extend(json.load(open(f)))

tags_df = pd.DataFrame(tags)
legit_tagged_sources = (
    tags_df
        .groupby('person')[['question_class', 'label']]
        .apply(lambda df: dict(df.itertuples(index=False)))
        .apply(pd.Series)
        .fillna('')
        .loc[lambda df: df['error'] == 'legit']
)

legit_tagged_sources = (
    legit_tagged_sources
        .reset_index()
        .assign(person=lambda df: df['person'].str.replace('-', ' '))
)

In [167]:
legit_tagged_sources.head()

,person,error,affiliation,gov-branch,gov-person-role,victim-person-role,academic-person-role,witness-person-role,actor-affiliation-type,actor-individual-person-role,actor-group-person-role,corporate-person-role,lawyer-role,ngo-person-role,stance,current
0,Adolfo Calero,legit,actor,,,,,,group,,leader,,,,informational,current
1,Adrian Ostfeld,legit,academic,,,,other,,,,,,,,informational,current
2,Alan Cranston,legit,government,legislative,decision-maker,,,,,,,,,,opposing,
3,Albert R. Brashear,legit,government,executive,spokesman,,,,,,,,,,supporting,
4,Allan Hoffenblum,legit,corporate,,,,,,,,,analyst,,,informational,current


In [168]:
tags_df.head()

,doc_id,label,person,question_class
0,84ba3ba0-c916-11e9-b04b-b831b5755f6c,legit,Junior-Bush,error
1,84ba3ba0-c916-11e9-b04b-b831b5755f6c,government,Junior-Bush,affiliation
2,84ba3ba0-c916-11e9-b04b-b831b5755f6c,executive,Junior-Bush,gov-branch
3,84ba3ba0-c916-11e9-b04b-b831b5755f6c,decision-maker,Junior-Bush,gov-person-role
4,84ba3ba0-c916-11e9-b04b-b831b5755f6c,NaN,Junior-Bush,victim-person-role


In [149]:
untagged_files = glob.glob('../app/data/*-unmarke*')

In [156]:
untagged_df = pd.concat([pd.DataFrame(json.load(open(f))) for f in untagged_files])

In [173]:
(legit_tagged_sources['person']
 .loc[lambda s: ~s.isin(untagged_df['person'])]
)

41          David Bar Illan
180    Sheila Evans Tranumn
205        Yossi Ben Aharon
Name: person, dtype: object

In [176]:
legit_tagged_sources_full = legit_tagged_sources.merge(untagged_df, how='left', left_on='person', right_on='person')

In [194]:
display(HTML(
    legit_tagged_sources_full
        .loc[lambda df: df['affiliation']=='ngo']
        .iloc[2]['html']
))

In [205]:
stanford_dir = "../data/news-article-flatlist/stanford-parses/"
stanford_parse_output = {}
for doc_id in (legit_tagged_sources_full
#                 .loc[lambda df: df['affiliation'].isin(['ngo', 'government', 'academic'])]
                .loc[:, 'doc_id']
                .dropna()
                .unique()
              ):
    stanford_parse_file = os.path.join(stanford_dir, doc_id) + '.txt.xml'
    sources, sents_list, named_ents_list,deps = parse_stanford_and_get_people(stanford_parse_file)
    stanford_parse_output[doc_id] = {
        "sources": sources,
        "sents_list": sents_list,
        "NER_list": named_ents_list,
        "deps_list": deps
    }

In [282]:
from collections import defaultdict

In [296]:
affiliation_to_orgs = defaultdict(set)
for source, doc_id, sentence_ids, affiliation in (
    legit_tagged_sources_full
             .loc[:, ['person', 'doc_id', 'sentence_ids', 'affiliation']]
             .dropna()
             .itertuples(index=False)
):
    for sentence_id in sentence_ids:
        sent_to_ents = list(zip(
            stanford_parse_output[doc_id]['sents_list'][sentence_id],
            stanford_parse_output[doc_id]['NER_list'][sentence_id]
        ))
        orgs = get_contiguous_organizations(sent_to_ents)
        affiliation_to_orgs[affiliation] |= set(orgs)
        break

In [323]:
source

'Yousef Sayigh'

In [326]:
stanford_parse_output[doc_id]['sources'][source]

{'mentions': [<Entity: {'name': 'Yousef Sayigh', 'start': 15, 'end': 16, 'head': 16, 'sentence': 28}>],
 'first_mention': <Entity: {'name': 'Yousef Sayigh', 'start': 15, 'end': 16, 'head': 16, 'sentence': 28}>,
 'speaking_vbs': [{'sentence': 28, 'verb_idx': 26}],
 'all_vbs': [{'sentence': 28, 'verb_idx': 26}]}

In [300]:
' '.join(stanford_parse_output[doc_id]['sents_list'][sentence_id])

"Any improvement in the economic situation will be felt by Hamas , too , '' Yousef Sayigh , a senior economic adviser to the P.L.O. , said at a seminar here on Tuesday ."

In [305]:
print(ET.tostring(stanford_parse_output[doc_id]['deps_list'][sentence_id][0]).decode('utf-8'))

<dep type="root">
            <governor idx="0">ROOT</governor>
            <dependent idx="27">said</dependent>
          </dep>
          


In [331]:
parsing_util.parse_dep(stanford_parse_output[doc_id]['deps_list'][sentence_id][0])

{'type': 'root',
 'governor': 'ROOT',
 'governor-idx': '0',
 'dependent': 'said',
 'dependent-idx': '27'}

In [327]:
stanford_parse_output[doc_id]['deps_list'][sentence_id][0].find('.//governor')
stanford_parse_output[doc_id]['deps_list'][sentence_id][0].find('.//dependent').attrib

{'idx': '27'}

In [394]:
exploded_df = (pd.Series(affiliation_to_orgs)
 .apply(list)
 .to_frame()
 .assign(len=lambda df: df[0].str.len())
 .assign(full_arr=lambda df: df.apply(lambda x: np.pad(np.array(x[0]), (0, df['len'].max()-x['len']), 'constant' ), axis=1))
 ['full_arr']
 .pipe(lambda s: pd.DataFrame(s.to_dict()))
 .replace('0', '')
)

In [400]:
pyperclip.copy(exploded_df.iloc[:8, 4:].to_latex(index=False))

In [354]:
pd.options.display.max_colwidth=500
pyperclip.copy(pd.Series(affiliation_to_orgs).apply(list).to_frame().T.iloc[:,:4].to_latex(escape=False, index=False))

In [402]:
yago_output = {}
for affil, org_list in tqdm(affiliation_to_orgs.items()):
    for org in tqdm(org_list):
        yago_output[org] = query_yago_for_organization(org)

In [406]:
pyperclip.copy((pd.Series(yago_output)
 .apply(lambda x: x.shape[0])
 .pipe(lambda s: s>0)
 .value_counts()
).to_latex())

(pd.Series(yago_output)
 .apply(lambda x: x.shape[0])
 .pipe(lambda s: s>0)
 .value_counts()
)

True     51
False    47
dtype: int64

In [412]:
pyperclip.copy((pd.Series(yago_output)
 .apply(lambda x: x.shape[0])
 .loc[lambda s: s==0].head(8).to_latex()
))

In [280]:
yago_mapping_flat = {}
for groupname, yago_out in yago_output.items():
    yago_mapping_flat[groupname] = (yago_out
                                    .get('val', pd.Series())
                                    .astype(str)
                                    .str.replace('http://www.w3.org/2002/07/', '')
                                    .str.replace('/', '')
                                    .unique().tolist()
                                   )

In [427]:
pd.Series(yago_mapping_flat).head()

Republican Party                  [owl#Thing, wikicat_Politi...
Esmor                                                        []
Legislature                                                  []
Yale University Medical School                               []
Palestinian Authority                                        []
dtype: object

1 kilobyte (KB) = 1,024 bytes

1 megabyte (MB) = 1,048,576 bytes

1 gigabyte (GB) = 1,073,741,824 bytes

In [439]:
1024 ** 3

1073741824

In [441]:
(pd.Series(glob.glob('D:/newspaper-front-page-project/yago/*'))
 .to_frame('file')
 .assign(fn=lambda df: df['file'].apply(os.path.basename))
 .assign(size=lambda df: df['file'].apply(lambda x: os.path.getsize(x) / (1024**3)).round(2))
)

,file,fn,size
0,D:/newspaper-front-page-pr...,yago3.1_entire_tsv.7z,18.04
1,D:/newspaper-front-page-pr...,yagoConteXtFacts_ar.tsv,0.76
2,D:/newspaper-front-page-pr...,yagoConteXtFacts_de.tsv,4.97
3,D:/newspaper-front-page-pr...,yagoConteXtFacts_en.tsv,15.53
4,D:/newspaper-front-page-pr...,yagoConteXtFacts_es.tsv,3.56
5,D:/newspaper-front-page-pr...,yagoConteXtFacts_fa.tsv,0.59
6,D:/newspaper-front-page-pr...,yagoConteXtFacts_fr.tsv,0.57
7,D:/newspaper-front-page-pr...,yagoConteXtFacts_it.tsv,3.62
8,D:/newspaper-front-page-pr...,yagoConteXtFacts_nl.tsv,2.42
9,D:/newspaper-front-page-pr...,yagoConteXtFacts_pl.tsv,2.46


In [454]:
yago_labels = pd.read_csv('D:/newspaper-front-page-project/yago/yagoLabels.tsv', sep='\t')

In [457]:
yago_labels.columns = ['id', 'entity', 'relation', 'label', 'useless']

In [459]:
yago_labels.head()

,id,entity,relation,label,useless
0,<id_fOrMgmH8cn_UVh_o6kCJ3c...,<Buffalo_Bill_Gun>,skos:prefLabel,Buffalo Bill Gun@eng,NaN
1,<id_Wb68!v3Sw4_UVh_Kaw39yg...,<Swimming_at_the_2012_Euro...,skos:prefLabel,Swimming at the 2012 Europ...,NaN
2,<id_rvwwOm6XOw_UVh_UgHXAnj...,<wordnet_superfecta_100507...,skos:prefLabel,superfecta@eng,NaN
3,<id_9kw08I9o?t_UVh_AD0Jd0e...,<Gornje_Biosko>,skos:prefLabel,Gornje Biosko@eng,NaN
4,<id_avA9jfopqk_UVh_Wy2d3mV...,<nl/Ondermarck_(familie)>,skos:prefLabel,Ondermarck (familie)@eng,NaN


In [464]:
del yago_labels

In [ ]:
yago_facts = pd.read_csv('D:/newspaper-front-page-project/yago/yagoFacts.tsv', sep='\t')

In [449]:
yago_facts.columns = ['id', 'predicate', 'relation', 'object', 'useless']

In [452]:
yago_facts['relation'].value_counts().head()

<isLocatedIn>       3074176
<isCitizenOf>       2141725
<hasGender>         1972842
<isAffiliatedTo>    1204540
<wasBornIn>          848846
Name: relation, dtype: int64